In [3]:
# Bidirectional RNN movie review

from keras.datasets import imdb
from keras.preprocessing import sequence
#from keras.preprocessing.sequence import pad_sequences
import numpy as np

max_features = 10000
# original:   maxlen=500
maxlen=1000

(x_train, y_train), (x_test, y_test) = imdb.load_data(
    num_words=max_features)

print(len(x_train), 'input train sequences')
print(len(y_test), 'test sequences')
print('x_train original shape: ', x_train.shape)

x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

''' what does pad_sequences really do?
does it reduce the vector-of-words size to 500(maxlen)??
x_train original shape:  (25000,) so it might vary in # of words??
Original comes as a compressed list of index values--obviously silly to use sparse singe-1 vectors.
I might try using 10K and improve my results!!'''
print('pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)

print('x_train shape after pad: ', x_train.shape)
print('x_test shape after pad: ', x_test.shape)

25000 input train sequences
25000 test sequences
x_train original shape:  (25000,)
pad sequences (samples x time)
x_train shape after pad:  (25000, 1000)
x_test shape after pad:  (25000, 1000)


In [8]:
from keras.models import Sequential
from keras import layers
from keras.layers import Embedding, LSTM, Dense
#from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Embedding(max_features, 128)) # this may not be the batch_size-128 below
# Tt seems that the digraph for bidirectional LSTM is built-in!          
model.add(layers.Bidirectional(  
    layers.LSTM(32)))
# 12-5 try adding another layer and see if I can beat 88% from on Bi layer  looks like 90%
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=3,  # original was 10 with overfitting starting at two.  Try 3 without validation
                    batch_size=128,
                    validation_split=0.2)  # validation_split allows me to set how many to train on--try 0!
                        # yep, default is 0.  validation_split=0.0, 

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 700s 35ms/step - loss: 0.5166 - acc: 0.7388 - val_loss: 0.3663 - val_acc: 0.8532
Epoch 2/3
20000/20000 [==============================] - 668s 33ms/step - loss: 0.3209 - acc: 0.8756 - val_loss: 0.4434 - val_acc: 0.8608
Epoch 3/3
20000/20000 [==============================] - 686s 34ms/step - loss: 0.2562 - acc: 0.9055 - val_loss: 0.5473 - val_acc: 0.7868


In [7]:
''' First: 20K training  started train at 12:15 about 6 min per epoch--expect 60 min? Done by 2:10
so less than 60 min.  
  Maybe the Dense 95% was overfitting?  So maybe I matched the result
Yes, I think I was overfitting even just before the 2nd epoch on Dense one, so real validation accuracy was < 91%.

I seem to have achieved 88% validation when I used 20% validation
    accuracy on movie review sentiments.  Chollet got 89% but 
    his training used another instance of random starting weights.
    
2.0    Re-ran bidirectional without using  any validation and got 88% again... or maybe 90%--touch better!
Epoch 2/3
25000/25000  780s 31ms/step - loss: 0.3101 - acc: 0.8822   <-- last non-overfit in original
Epoch 3/3
25000/25000 772s 31ms/step - loss: 0.2665 - acc: 0.9010
Second without validation:  started 25000 x 1000 training set at 9:40 PM. 
No validation--let's see how high we get on acc. after 2nd epoch!  Goal beat 95.7 from
Dense w/o validation!    

    
3.0    Compare one-way RNN with 88% and
    original 2, Dense, 16 channel layers I got 89%?!
    
    In fact in the original Dense(16) X2 layer training when I re-ran for just 4 epochs I got a whopping
    95% Can you still accept this?  
    I think so--It looks like I used 5000 more training samples in the 
    final, Dense-only training.  
    
    I skipped all validation during training since I validated in the
    previous run.  Could try this here with the bidirectional and see how well I can do!
    And I used 10K size word vecs in Densely Connected model vs 5 K here!  Also for Dense I used
    optimizers.RMSprop(lr=0.001)
    vs 'rmsprop' here...
    val_binary_accuracy: 0.9577
fit complete
25000/25000 [==============================] - 1s 42us/step
results:  [0.28607536478996276, 0.8856]

    When I re-ran with 4 ep
    
    Try more layers??
    4.0 no luck with another bidir 32 layer--exception.
 5.  Try adding a Dense layer as in 95% 3 Dense training   
    model.add(layers.Dense(16, activation='relu'))
    5.1 start around 7:45  est. 45 minutes
'''
import matplotlib.pyplot as plt

acc = history.history['acc']
validation_acc = history.history['val_acc']
loss = history.history['loss']
validation_loss = history.history['val_loss']
#print("validation_loss: ", validation_loss) 

epochs = range(1,len(acc) + 1) # list for x axis

plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, validation_acc, 'b', label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')

plt.ylabel('Loss')
plt.legend()
plt.show()

KeyError: 'val_acc'